In [1]:
!pip install --quiet -r requirements.txt

In [2]:
!nvidia-smi

Sat Jan 11 04:07:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060        Off |   00000000:01:00.0  On |                  N/A |
|  0%   44C    P3             32W /  170W |     862MiB /  12288MiB |     27%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import warnings
warnings.filterwarnings('ignore')

import os, gc
import pandas as pd
import polars as pl
import random
import psutil
import xgboost as xgb
#import catboost as cbt # needs numpy <2.0
import numpy as np
#from hyperopt import hp, fmin, tpe, Trials
#from hyperopt.pyll.base import scope
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
#from tqdm.notebook import tqdm
#from joblib import dump, load
#import datatable as dtable
#from mlxtend.evaluate import GroupTimeSeriesSplit
import kaggle_evaluation.jane_street_inference_server as js_server

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
# check that torch is working and sees the GPU
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
torch.set_default_dtype(torch.float16) # because memory is fucking huge and I have to do this

2.4.1+cu121
True
NVIDIA GeForce RTX 3060


In [5]:
collected = gc.collect()
# Prints Garbage collector 
# as 0 object
print("Garbage collector: collected",
          "%d objects." % collected)

Garbage collector: collected 72 objects.


# Globals and prep

In [6]:
PATH = os.getcwd() + '/input/'

METAS = ['date_id', 'time_id', 'symbol_id', 'weight']
FEATURES = [f'feature_{i:02}' for i in range(79)]
RESPONDERS = [f'responder_{i}' for i in range(9)]
RESPONDERS_LAGS = [f'responder_{i}_lag_1' for i in range(9)]
TARGET = 'responder_6'
FEATURES_AND_LAGS=True
ONLY_TARGET_LAGS=True
if FEATURES_AND_LAGS and not ONLY_TARGET_LAGS:
    FEATURES_WORKING = FEATURES + RESPONDERS + RESPONDERS_LAGS
    FEATURES_WORKING.remove(TARGET)
elif FEATURES_AND_LAGS and ONLY_TARGET_LAGS:
    FEATURES_WORKING = FEATURES + ['responder_6_lag_1']
else:
    FEATURES_WORKING = FEATURES
print(FEATURES_WORKING)

USEFUL_COLS = FEATURES_WORKING + [TARGET] + ['weight']

SEED = 728

['feature_00', 'feature_01', 'feature_02', 'feature_03', 'feature_04', 'feature_05', 'feature_06', 'feature_07', 'feature_08', 'feature_09', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20', 'feature_21', 'feature_22', 'feature_23', 'feature_24', 'feature_25', 'feature_26', 'feature_27', 'feature_28', 'feature_29', 'feature_30', 'feature_31', 'feature_32', 'feature_33', 'feature_34', 'feature_35', 'feature_36', 'feature_37', 'feature_38', 'feature_39', 'feature_40', 'feature_41', 'feature_42', 'feature_43', 'feature_44', 'feature_45', 'feature_46', 'feature_47', 'feature_48', 'feature_49', 'feature_50', 'feature_51', 'feature_52', 'feature_53', 'feature_54', 'feature_55', 'feature_56', 'feature_57', 'feature_58', 'feature_59', 'feature_60', 'feature_61', 'feature_62', 'feature_63', 'feature_64', 'feature_65', 'feature_66', 'feature_67', 'feature_68', 'feature_69', 'feature_70', 'feat

In [7]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # Ensure deterministic behavior (may impact performance)
        #torch.backends.cudnn.deterministic = False
        #torch.backends.cudnn.benchmark = False

def lazy_load(par_path):
    return pl.scan_parquet(par_path).select(
        pl.int_range(pl.len(), dtype=pl.UInt64).alias("index"),
        pl.all()
    )
seed_everything(SEED) # just do it

In [22]:
# for the purged group time series split, code is copied from somewhere
# TODO: make GitHub GIST

# TODO: add as dataset
# TODO: add logging with verbose

from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.|
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [ ]:
# from matplotlib.colors import ListedColormap
    
# # this is code slightly modified from the sklearn docs here:
# # https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#sphx-glr-auto-examples-model-selection-plot-cv-indices-py
# def plot_cv_indices(cv, X, y, group, ax, n_splits, lw=10):
#     """Create a sample plot for indices of a cross-validation object."""
    
#     cmap_cv = plt.cm.coolwarm

#     jet = plt.cm.get_cmap('jet', 256)
#     seq = np.linspace(0, 1, 256)
#     _ = np.random.shuffle(seq)   # inplace
#     cmap_data = ListedColormap(jet(seq))

#     # Generate the training/testing visualizations for each CV split
#     for ii, (tr, tt) in enumerate(cv.split(X=X, y=y, groups=group)):
#         # Fill in indices with the training/test groups
#         indices = np.array([np.nan] * len(X))
#         indices[tt] = 1
#         indices[tr] = 0

#         # Visualize the results
#         ax.scatter(range(len(indices)), [ii + .5] * len(indices),
#                    c=indices, marker='_', lw=lw, cmap=cmap_cv,
#                    vmin=-.2, vmax=1.2)

#     # Plot the data classes and groups at the end
#     ax.scatter(range(len(X)), [ii + 1.5] * len(X),
#                c=y, marker='_', lw=lw, cmap=plt.cm.Set3)

#     ax.scatter(range(len(X)), [ii + 2.5] * len(X),
#                c=group, marker='_', lw=lw, cmap=cmap_data)

#     # Formatting
#     yticklabels = list(range(n_splits)) + ['target', 'day']
#     ax.set(yticks=np.arange(n_splits+2) + .5, yticklabels=yticklabels,
#            xlabel='Sample index', ylabel="CV iteration",
#            ylim=[n_splits+2.2, -.2], xlim=[0, len(y)])
#     ax.set_title('{}'.format(type(cv).__name__), fontsize=15)
#     return ax


# fig, ax = plt.subplots()
# #plot_cv_indices(cv, X_train, y_train, groups, ax, 5, lw=20)
# plot_cv_indices(
#     cv,
#     train[features].values,
#     train['responder_6'].values,
#     train['date_id'].values,
#     ax,
#     5,
#     lw=20
# )
# rubbish=gc.collect()

# code structure copied/inspired from https://www.kaggle.com/code/shiyili/js2024-rmf-mlp-inference-pytorch

In [23]:
### Define various helper functions including r2 score
writer = SummaryWriter()
# get cpu, gpu or mps device for training
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

def reset_weights(m):
    '''
    Try resetting model weights to avoid
    weight leakage.
    '''
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()

# for monitoring layer weights
def weight_histograms(writer, step, model):
    print("Visualizing model weights...")
    # Iterate over all model layers
    for layer_number in range(len(model.layers)):
        layer = model.layers[layer_number]
        try:
            weights = layer.weight
            flattened_weights = weights.flatten()
            tag = f"layer_{layer_number}"
            writer.add_histogram(tag, flattened_weights, global_step=step, bins='tensorflow')
        except AttributeError:
            return
            
# loss is as defined on competition homepage
# score will be = 1 - loss
def r2_loss(outputs, targets, weights):
    loss = torch.sum(weights*(targets - outputs)**2) / (torch.sum(weights*targets**2)+1e-38)
    return loss

# "standard" loss function
test_loss_function= nn.MSELoss()

Using cuda device


In [34]:
# Basic LSTM model
# class LSTM(nn.Module):
#     def __init__(self, input_dim, output_dim, hidden_dim):
#         super().__init__()
#         self.input_dim = input_dim
#         self.hidden_dim = hidden_dim
#         self.output_dim = output_dim

#         self.lstm = nn.LSTM(input_size=self.input_dim, hidden_size=self.hidden_dim, num_layers=1, batch_first=True)
#         self.linear = nn.Linear(self.hidden_dim, self.output_dim)
        
#     def forward(self, x):
#         '''Forward pass'''
#         x, _ = self.lstm(x)
#         x = self.linear(x)
#         return x.squeeze()
# basic NN model
class MLP(nn.Module):
    '''
    Simple Neural Network/MultiLayer Perceptron
    '''
    def __init__(self, input_size, hidden_size, output_size, dropout_rates):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_rates = dropout_rates
        current_size = input_size
        self.layers = nn.ModuleList()
        self.layers.append(nn.BatchNorm1d(current_size))
        self.layers.append(nn.Dropout(dropout_rates[0]))
        for i in range(len(hidden_size)):
            self.layers.append(nn.Linear(current_size, self.hidden_size[i]))
            self.layers.append(nn.BatchNorm1d(self.hidden_size[i]))
            self.layers.append(nn.SiLU())
            self.layers.append(nn.Dropout(self.dropout_rates[i+1]))
            current_size = self.hidden_size[i]
        self.layers.append(nn.Linear(current_size, self.output_size))

        self.seq = nn.Sequential(*self.layers)
        
    def forward(self, x):
        '''Forward pass'''
        return self.seq(x)

# copied LSTM equivalent model
class LSTMEquavalentMLP(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        super(LSTMEquavalentMLP, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.input_gate = nn.Linear(input_size, hidden_size)
        self.candidate_gate = nn.Linear(input_size, hidden_size)
        self.output_gate = nn.Linear(input_size, hidden_size)
        
    def forward(self, x):
        
        i_t = torch.sigmoid(self.input_gate(x)) # input gate
        c_t = torch.tanh(self.candidate_gate(x)) # candidate gate
        o_t = torch.sigmoid(self.output_gate(x)) # output gate
        
        h_t = o_t * torch.tanh(c_t * i_t)
        
        return h_t
    
class LSTMFeedForward(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size, dropout_rates=0):
        super(LSTMFeedForward, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.lstm = LSTMEquavalentMLP(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        h_t = self.lstm(x)
        y = self.fc(h_t).squeeze()
        return y

In [42]:
# training function
def train(model, trainloader, optimizer, num_epochs, writer, gradclip=1.0, SAVE=True):
    print_loss_every = 3000
    model.train()
    batch_train_tally = 0
    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):
        # Visualize weight histograms
        #weight_histograms(writer, epoch, model)
        # Print epoch
        print(f'Starting epoch {epoch+1}')
        
        # Set current loss value
        current_loss = 0.0
        # Iterate over Dataloader for training data
        for i, data in enumerate(trainloader):
            model.half()
            # Get inputs
            inputs, targets, weights = data
            # GPU must see the data and the model
            inputs = inputs.to(device) 
            targets = targets.to(device)
            weights = weights.to(device)
            
            # Zero the gradients
            optimizer.zero_grad()
            
            # Perform forward pass
            outputs = model(inputs)
            
            # Compute loss
            loss = r2_loss(outputs, targets, weights)
            writer.add_scalar("Loss/train/minibatches", loss, batch_train_tally)
            batch_train_tally += 1
            current_loss += loss.item()
            if i % print_loss_every == 0:
                print('Loss after mini-batch %5d: %.3f' %
                      (i + 1, current_loss / print_loss_every))
                current_loss = 0.0
            # Perform backward pass
            loss.backward()
        
            for name, param in model.named_parameters():
                if param.grad is not None:
                    if torch.isnan(param.grad).any():
                        print(f"NaN found in the gradient of {name}")
                        
            # gradient clipping to try to stop explosions
            torch.nn.utils.clip_grad_norm_(model.parameters(), gradclip)
            model.float() # update weights using regular precision (else you get nans when working with half data)
            # Update weights
            optimizer.step()

        # Finished iterating over Dataloader
        writer.add_scalar("Loss/train/epoch", loss, epoch)
        writer.flush()

        # save after every epoch
        if SAVE:
            save_path = f'./models/lstm_e{epoch}.pth'
            torch.save(model.state_dict(), save_path)
      
    # Saving the final model
    if SAVE:
        save_path = f'./models/working_lstm_f{fold}.pth'
        torch.save(model.state_dict(), save_path)
    return

# testing function
def test(model, testloader, writer):
    # Evaluation for this fold
    batch_test_tally = 0
    R2_score = 0.0
    total_loss = 0.0
    with torch.no_grad():
        model.eval()
        # Iterate over the test data and generate predictions
        for i, data in enumerate(testloader):
            model.half()
            # Get inputs
            inputs, targets, weights = data
            inputs = inputs.to(device) # GPU must see the data and the model
            targets = targets.to(device)
            weights = weights.to(device)
        
            # Generate outputs
            outputs = model(inputs)
            loss = r2_loss(outputs, targets, weights)
            if i%1000==0:
                print(f'i={i}, loss={loss}')
            writer.add_scalar('Loss/test/minibatches', loss, batch_test_tally)
            batch_test_tally += 1
            total_loss += loss
        return total_loss

# training function
def train_and_test(model, trainloader, testloader, optimizer, num_epochs, writer, gradclip=1.0, SAVE=True, model_type='type'):
    print_loss_every = 3000
    val_every = 1
    batch_train_tally = 0
    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):
        model.train()
        # Visualize weight histograms
        #weight_histograms(writer, epoch, model)
        # Print epoch
        print(f'Starting epoch {epoch+1}')
        
        # Set current loss value
        current_loss = 0.0
        # Iterate over Dataloader for training data
        for i, data in enumerate(trainloader):
            model.half()
            # Get inputs
            inputs, targets, weights = data
            # GPU must see the data and the model
            inputs = inputs.to(device) 
            targets = targets.to(device)
            weights = weights.to(device)
            
            # Zero the gradients
            optimizer.zero_grad()
            
            # Perform forward pass
            outputs = model(inputs)
            
            # Compute loss
            loss = r2_loss(outputs, targets, weights)
            writer.add_scalar("Loss/train/minibatches", loss, batch_train_tally)
            batch_train_tally += 1
            current_loss += loss.item()
            if i % print_loss_every == 0:
                print('Loss after mini-batch %5d: %.3f' %
                      (i + 1, current_loss / print_loss_every))
                current_loss = 0.0
            # Perform backward pass
            loss.backward()
        
            for name, param in model.named_parameters():
                if param.grad is not None:
                    if torch.isnan(param.grad).any():
                        print(f"NaN found in the gradient of {name}")
                        
            # gradient clipping to try to stop explosions
            torch.nn.utils.clip_grad_norm_(model.parameters(), gradclip)
            model.float() # update weights using regular precision (else you get nans when working with half data)
            # Update weights
            optimizer.step()

        # Finished iterating over Dataloader
        writer.add_scalar("Loss/train/epoch", loss, epoch)
        writer.flush()

        # save after every epoch
        if SAVE:
            save_path = f'./models/{model_type}_e{epoch}.pth'
            torch.save(model.state_dict(), save_path)


        # validate after every epoch
        if epoch % val_every == 0:
            # Evaluation for this epoch
            batch_test_tally = 0
            R2_score = 0.0
            total_loss = 0.0
            with torch.no_grad():
                model.eval()
                # Iterate over the test data and generate predictions
                for i, data in enumerate(testloader):
                    model.half()
                    # Get inputs
                    inputs, targets, weights = data
                    inputs = inputs.to(device) # GPU must see the data and the model
                    targets = targets.to(device)
                    weights = weights.to(device)
                
                    # Generate outputs
                    outputs = model(inputs)
                    loss = r2_loss(outputs, targets, weights)
                    # if i%1000==0:
                    #     print(f'i={i}, loss={loss}')
                    writer.add_scalar('Loss/test/minibatches', loss, batch_test_tally)
                    batch_test_tally += 1
                    total_loss += loss
                    num_batches = len(testloader)
                    results = total_loss / num_batches
                    writer.add_scalar('Loss/test/epoch', results, epoch)
                    writer.flush()
                print(f'Validation score after {epoch} epochs={results}')
                #return total_loss
      
    # Saving the final model
    if SAVE:
        save_path = f'./models/working_{model_type}_f{fold}.pth'
        torch.save(model.state_dict(), save_path)
    return


In [11]:
print('Loading trainset...')
TEST=False
if TEST:
    trainset = torch.load(f'./input/trainsets/working_trainset_TEST.pt')
else:
    trainset = torch.load(f'./input/trainsets/working_trainset.pt')
print('Loaded.')

Loading trainset...
Loaded.


In [12]:
print('Loading validset...')
if TEST:
    validset = torch.load(f'./input/validsets/working_validset_TEST.pt')
else:
    validset = torch.load(f'./input/validsets/working_validset.pt')
print('Loaded.')

Loading validset...
Loaded.


In [39]:
LSTM_FF_CONFIG = {
    'input_size':  len(trainset[-1][0]),
    'hidden_size': 128,
    'output_size': 1,
    'dropout_rates': 0
}

MLP_CONFIG = {
    'input_size': len(trainset[-1][0]),
    'hidden_size': [128, 256, 256, 128, 64],
    'output_size': 1,
    'dropout_rates': [0.1, 0.1, 0.1, 0.1,0.1,0.1]
}

TRAINING_CONFIG = {
    'num_epochs': 10,
    'learning_rate': 1e-4,
    'grad_clip':1.0,
    'test_loss': False
}

fold_function = 'PurgedGroupTimeSeries'
# fold_function = 'KFold'
fold_function = 'None'

# choose a model
model_type = 'LSTM_FF' # 'LSTM_FF' or 'NN'
# For holding fold results
results = {}

In [32]:
# Define data loaders
BATCH_SIZE = 2048
NUM_WORKERS = 4 # num of parallel subprocesses for data loading (CPU task)
trainloader = torch.utils.data.DataLoader(
    trainset,
    batch_size=BATCH_SIZE,
    num_workers = NUM_WORKERS
)
testloader = torch.utils.data.DataLoader(
    validset,
    batch_size=1,
    num_workers = NUM_WORKERS
)

In [43]:
import datetime
now = datetime.datetime.now()
print(f'Started on {now}')
### Train and cross validation loop
folds = [([0,0],[0,0])]
for fold, (train_ids, test_ids) in enumerate(folds):
    # Print
    print(f'FOLD {fold}')
    print(f'train_ids: [{train_ids[0]}, {train_ids[-1]}]')
    print(f'test_ids: [{test_ids[0]}, {test_ids[-1]}]')
    if fold_function != 'None':
        print("Using a fold function...")
        # define train/test sets
        X_tr, y_tr, w_tr = X[train_ids], y[train_ids], w[train_ids]
        #X_te, y_te, w_te = X[test_ids], y[test_ids], w[test_ids]
    
    print('--------------------------------')

    # Initialize Dataset objects to make PyTorch play nice
    #trainset = TensorDataset(torch.tensor(X_tr).to(torch.float32), torch.tensor(y_tr), torch.tensor(w_tr))
    #testset = TensorDataset(torch.tensor(X_te).to(torch.float32), torch.tensor(y_te), torch.tensor(w_te))
    #trainset = TensorDataset(X_tr, y_tr, w_tr)
    #testset = TensorDataset(X_te, y_te, w_te)

    # Init the model
    if model_type == 'NN':
        model = MLP(**MLP_CONFIG).to(device)
    elif model_type == 'LSTM_FF' :
        model = LSTMFeedForward(**LSTM_FF_CONFIG).to(device)
    model.apply(reset_weights) # not sure if I need this but w/e
    # Initialize optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=TRAINING_CONFIG['learning_rate'])
    BATCH_TRAIN_TALLY = 0 # global var lmao
    BATCH_TEST_TALLY = 0
    train_and_test(model, trainloader, testloader, optimizer, TRAINING_CONFIG["num_epochs"], writer, gradclip=TRAINING_CONFIG["grad_clip"], SAVE=False, model_type=model_type)
    # Process is complete.
    print('Training process has finished')
    print('Deleting the training data variables...')
    del trainset, trainloader
    gc.collect()
    
writer.close()
rubbish=gc.collect()
now = datetime.datetime.now()
print(f'Ended on {now}')

Started on 2025-01-11 04:47:49.340231
FOLD 0
train_ids: [0, 0]
test_ids: [0, 0]
--------------------------------
Reset trainable parameters of layer = Linear(in_features=80, out_features=128, bias=True)
Reset trainable parameters of layer = Linear(in_features=80, out_features=128, bias=True)
Reset trainable parameters of layer = Linear(in_features=80, out_features=128, bias=True)
Reset trainable parameters of layer = Linear(in_features=128, out_features=1, bias=True)
Starting epoch 1
Loss after mini-batch     1: 0.000
Loss after mini-batch  3001: 0.997
Loss after mini-batch  6001: 0.996
Loss after mini-batch  9001: 0.995
Loss after mini-batch 12001: 0.993
Loss after mini-batch 15001: 0.992
Loss after mini-batch 18001: 0.988


KeyboardInterrupt: 

In [ ]:
import datetime
now = datetime.datetime.now()
print(f'Started on {now}')
### Train and cross validation loop
folds = [([0,0],[0,0])]
for fold, (train_ids, test_ids) in enumerate(folds):
    # Print
    print(f'FOLD {fold}')
    print(f'train_ids: [{train_ids[0]}, {train_ids[-1]}]')
    print(f'test_ids: [{test_ids[0]}, {test_ids[-1]}]')
    if fold_function != 'None':
        print("Using a fold function...")
        # define train/test sets
        X_tr, y_tr, w_tr = X[train_ids], y[train_ids], w[train_ids]
        #X_te, y_te, w_te = X[test_ids], y[test_ids], w[test_ids]
    
    print('--------------------------------')

    # Initialize Dataset objects to make PyTorch play nice
    #trainset = TensorDataset(torch.tensor(X_tr).to(torch.float32), torch.tensor(y_tr), torch.tensor(w_tr))
    #testset = TensorDataset(torch.tensor(X_te).to(torch.float32), torch.tensor(y_te), torch.tensor(w_te))
    #trainset = TensorDataset(X_tr, y_tr, w_tr)
    #testset = TensorDataset(X_te, y_te, w_te)

    # Init the model
    model = LSTMFeedForward(**LSTM_FF_CONFIG).to(device)
    model.apply(reset_weights) # not sure if I need this but w/e
    # Initialize optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=TRAINING_CONFIG['learning_rate'])
    BATCH_TRAIN_TALLY = 0 # global var lmao
    train(model, trainloader, optimizer, TRAINING_CONFIG["num_epochs"], writer, gradclip=TRAINING_CONFIG["grad_clip"], SAVE=True)
    # Process is complete.
    print('Training process has finished')
    print('Deleting the training data variables...')
    del trainset, trainloader
    gc.collect()
    
    # Print about testing
    print('Starting testing')
    print('Loading validset...')
    if TEST:
        validset = torch.load(f'./input/validsets/working_validset_TEST.pt')
    else:
        validset = torch.load(f'./input/validsets/working_validset.pt')
    print('Loaded.')
    testloader = torch.utils.data.DataLoader(
        validset,
        batch_size=BATCH_SIZE,
        num_workers = NUM_WORKERS
    )
    
    BATCH_TEST_TALLY = 0 # global var lmao
    total_loss = test(model, testloader, writer)
    num_batches = len(testloader)
    results[fold] = total_loss / num_batches
    writer.add_scalar('Loss/test/fold', results[fold], fold)
    writer.flush()
    del validset, testloader

    # Print fold results
    print(f'CROSS VALIDATION RESULTS FOR {folds} FOLDS')
    print('--------------------------------')
    sum = 0.0
    for key, value in results.items():
        print(f'Fold {key}: {value}')
        sum += value
    print(f'Average: {sum/len(results.items())}')
writer.close()
rubbish=gc.collect()
now = datetime.datetime.now()
print(f'Ended on {now}')

In [15]:
gc.collect()

20

# Arbitrarily testing model after certain epoch

In [16]:
epochtest = 70
model = torch.load(f'./models/lstm_e{epochtest}.pth')
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model:
    print(param_tensor, "\t", model[param_tensor].size())

Model's state_dict:
lstm.input_gate.weight 	 torch.Size([128, 80])
lstm.input_gate.bias 	 torch.Size([128])
lstm.candidate_gate.weight 	 torch.Size([128, 80])
lstm.candidate_gate.bias 	 torch.Size([128])
lstm.output_gate.weight 	 torch.Size([128, 80])
lstm.output_gate.bias 	 torch.Size([128])
fc.weight 	 torch.Size([1, 128])
fc.bias 	 torch.Size([1])


In [17]:
loadedmodel = LSTMFeedForward(**LSTM_FF_CONFIG)
epochtest = 3
loadedmodel.load_state_dict(torch.load(f'./models/lstm_e{epochtest}.pth', weights_only=True))
loadedmodel.to(device)
# Print about testing
print('Starting testing')
print('Loading validset...')
if TEST:
    validset = torch.load(f'./input/validsets/working_validset_TEST.pt')
else:
    validset = torch.load(f'./input/validsets/working_validset.pt')
print('Loaded.')
testloader = torch.utils.data.DataLoader(
    validset,
    batch_size=int(BATCH_SIZE/2),
    num_workers = NUM_WORKERS
)

BATCH_TEST_TALLY = 0 # global var lmao
total_loss = test(loadedmodel, testloader, writer)
num_batches = len(testloader)
results[fold] = total_loss / num_batches
print(f'total_loss={total_loss}, num_batches={num_batches}')
#writer.add_scalar('Loss/test/fold', results[fold], fold)
#writer.flush()
#del validset, testloader

# # Print fold results
# print(f'CROSS VALIDATION RESULTS FOR {folds} FOLDS')
# print('--------------------------------')
# valsum = 0.0
# for key, value in results.items():
#     print(f'Fold {key}: {value}')
#     valsum += value
# print(f'Average: {valsum/len(results.items())}')

Starting testing
Loading validset...
Loaded.
i=0, loss=1.0107421875
i=1000, loss=0.923828125
i=2000, loss=1.0
i=3000, loss=0.951171875
i=4000, loss=1.03515625
i=5000, loss=0.978515625
i=6000, loss=1.015625
i=7000, loss=1.048828125
i=8000, loss=1.0322265625
i=9000, loss=1.046875


NameError: name 'fold' is not defined

In [ ]:
total_loss

In [77]:
model1 = LSTMFeedForward(**LSTM_FF_CONFIG)
epochtest = 1
model1.load_state_dict(torch.load(f'./models/lstm_e{epochtest}.pth', weights_only=True))

model2 = LSTMFeedForward(**LSTM_FF_CONFIG)
epochtest = 11
model2.load_state_dict(torch.load(f'./models/lstm_e{epochtest}.pth', weights_only=True))


<All keys matched successfully>

In [100]:
x_test = np.arange(0, len(FEATURES)+1, 1)
model2.to(device)
model2.half()
with torch.no_grad():
    model2.eval()
    x_test = torch.from_numpy(x_test)
    x_test = x_test.half().to(device)
    predict = model2(x_test).cpu().numpy()

predict

array(1.178, dtype=float16)

In [101]:
predict

array(1.178, dtype=float16)

In [19]:
for param_tensor in model1.state_dict():
    print(param_tensor, "\t", model1.state_dict()[param_tensor].size())

lstm.input_gate.weight 	 torch.Size([128, 80])
lstm.input_gate.bias 	 torch.Size([128])
lstm.candidate_gate.weight 	 torch.Size([128, 80])
lstm.candidate_gate.bias 	 torch.Size([128])
lstm.output_gate.weight 	 torch.Size([128, 80])
lstm.output_gate.bias 	 torch.Size([128])
fc.weight 	 torch.Size([1, 128])
fc.bias 	 torch.Size([1])


In [ ]:
model1.state_dict()['fc.bias']

In [ ]:
model2.state_dict()['fc.bias']

In [ ]:
validset[0][0].shape

# Run Inference

In [121]:
def predict_func(x, models):
    result_all = np.zeros((len(models), len(x)))
    with torch.no_grad():
        x = torch.tensor(x, dtype=torch.float).to(DEVICE)
        for i, model in enumerate(models):    
            predict = model(x).cpu().numpy()
            result_all[i, :] = predict
    return result_all.mean(axis=0)
       
def predict_func(x, model):
    with torch.no_grad():
        model.eval()
        x = torch.tensor(x).half().to(device)
        result = model(x).cpu().numpy()
        return result


In [158]:
class JaneStreetPredictor:
    
    def __init__(self, model, test_parquet, lag_parquet, sequence_len, feature_cols, pbar_length=0):
        # Initialize model and parameters
        self.model = model
        self.sequence_len = sequence_len
        self.feature_cols = feature_cols
        self.responder_cols = [f'responder_{i}' for i in range(9)]
        
        # Initialize parquet data for test and lag
        self.test_parquet = test_parquet
        self.lag_parquet = lag_parquet
        
        # Initialize global variables as class attributes
        self.history_cache = {}
        self.test_ = None
        self.lags_ = None
        self.time_step_count = 0
        # setup pbar:
        #self.pbar = tqdm(total=pbar_length, disable=(pbar_length == 0))

    

    def predict(self, test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
        
        # Handle lags if provided
        if lags is not None:
            self.lags_ = lags.to_pandas()
            self.lag_stats = self.lags_.groupby('symbol_id')[[f'{col}_lag_1' for col in self.responder_cols]].agg([
                'mean', 'std', 'max', 'min', lambda x: pd.Series(x).skew(), lambda x: pd.Series(x).kurt(), 
            ]).rename(columns={'<lambda_0>': 'skew', '<lambda_1>': 'kurt'})
            
            self.lag_stats.columns =  [f"{'_'.join(c1.split('_')[:-1])}_{c2}" for (c1, c2) in self.lag_stats.columns ]

        # Cache the test data for future use
        self.test_ = test

        test = test.to_pandas()
        test = test.fillna(0.0)
        test['responder_6_lag_1'] = self.lags_['responder_6_lag_1']
        print(test['responder_6_lag_1'][0:4])
        x_stack = test[self.feature_cols]
        print(x_stack.head())
        #x_stack = train_scaler.transform(test[self.feature_cols])
        #test = pd.merge(test, self.lag_stats, on='symbol_id', how='left')
        print(test.columns)
        x_stack = x_stack.values
       
        # Call the model's prediction function
        predicts = predict_func(x_stack, self.model)  # Placeholder for the actual model prediction logic

        # Create a DataFrame to store the predictions
        predictions = pl.DataFrame({'row_id': test['row_id'].values, 'responder_6': predicts})

        # sanity check
        assert isinstance(predictions, pl.DataFrame | pd.DataFrame)
        assert list(predictions.columns) == ['row_id', 'responder_6']
        assert len(predictions) == len(test)

        # update time_step_count
        self.time_step_count += 1
        #self.pbar.update(1)

        return predictions

    def run_inference_server(self):
    
        #self.pbar.refresh()

        inference_server = js_server.JSInferenceServer(self.predict)
        
        if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
            inference_server.serve()
        else:
            inference_server.run_local_gateway((self.test_parquet, self.lag_parquet))

        #self.pbar.close()


In [159]:
model_eval = LSTMFeedForward(**LSTM_FF_CONFIG)
model_eval.load_state_dict(torch.load(f'./models/working_lstm_f0.pth', weights_only=True))
model_eval.to(device)
test_parquet = os.getcwd() + '/input/test.parquet'
lags_parquet = os.getcwd() + '/input/lags.parquet'
SEQUENCE_LEN=16
js_predictor = JaneStreetPredictor(
    model_eval,
    test_parquet,
    lags_parquet,
    sequence_len= SEQUENCE_LEN,
    feature_cols = FEATURES_WORKING,
)

js_predictor.run_inference_server()

0   -1.305746
1   -1.162801
2   -1.574290
3    0.329152
Name: responder_6_lag_1, dtype: float32
   feature_00  feature_01  feature_02  feature_03  feature_04  feature_05  \
0         0.0         0.0         0.0         0.0         0.0        -0.0   
1         0.0        -0.0         0.0         0.0         0.0        -0.0   
2         0.0        -0.0         0.0         0.0         0.0        -0.0   
3         0.0         0.0         0.0         0.0         0.0        -0.0   
4         0.0        -0.0         0.0         0.0         0.0        -0.0   

   feature_06  feature_07  feature_08  feature_09  ...  feature_70  \
0        -0.0        -0.0         0.0         0.0  ...        -0.0   
1        -0.0        -0.0         0.0         0.0  ...        -0.0   
2        -0.0        -0.0         0.0         0.0  ...        -0.0   
3        -0.0        -0.0         0.0         0.0  ...        -0.0   
4        -0.0        -0.0         0.0         0.0  ...        -0.0   

   feature_71  featu

In [170]:
js_predictor.run_inference_server()

0   -1.305746
1   -1.162801
2   -1.574290
3    0.329152
Name: responder_6_lag_1, dtype: float32
   feature_00  feature_01  feature_02  feature_03  feature_04  feature_05  \
0         0.0         0.0         0.0         0.0         0.0        -0.0   
1         0.0        -0.0         0.0         0.0         0.0        -0.0   
2         0.0        -0.0         0.0         0.0         0.0        -0.0   
3         0.0         0.0         0.0         0.0         0.0        -0.0   
4         0.0        -0.0         0.0         0.0         0.0        -0.0   

   feature_06  feature_07  feature_08  feature_09  ...  feature_70  \
0        -0.0        -0.0         0.0         0.0  ...        -0.0   
1        -0.0        -0.0         0.0         0.0  ...        -0.0   
2        -0.0        -0.0         0.0         0.0  ...        -0.0   
3        -0.0        -0.0         0.0         0.0  ...        -0.0   
4        -0.0        -0.0         0.0         0.0  ...        -0.0   

   feature_71  featu

In [171]:
if os.path.isfile('submission.parquet'):
    pl_sub = pl.read_parquet('submission.parquet')
    print(len(pl_sub))
    display(pl_sub)

39


row_id,responder_6
i64,f32
0,0.050659
1,0.027863
2,0.093018
3,-0.231567
4,-0.210083
…,…
34,-0.070068
35,-0.27002
36,0.332764


In [157]:
if os.path.isfile('submission.parquet'):
    pl_sub = pl.read_parquet('submission.parquet')
    print(len(pl_sub))
    display(pl_sub)

39


row_id,responder_6
i64,f32
0,0.050659
1,0.027863
2,0.093018
3,-0.231567
4,-0.210083
…,…
34,-0.070068
35,-0.27002
36,0.332764


In [142]:
js_predictor.lags_

,date_id,time_id,symbol_id,responder_0_lag_1,responder_1_lag_1,responder_2_lag_1,responder_3_lag_1,responder_4_lag_1,responder_5_lag_1,responder_6_lag_1,responder_7_lag_1,responder_8_lag_1
0,0,0,0,-0.442215,-0.322407,0.143594,-0.926890,-0.782236,-0.036595,-1.305746,-0.795677,-0.143724
1,0,0,1,-0.651829,-1.707840,-0.893942,-1.065488,-1.871338,-0.615652,-1.162801,-1.205924,-1.245934
2,0,0,2,-0.656373,-0.264575,-0.892879,-1.511886,-1.033480,-0.378265,-1.574290,-1.863071,-0.027343
3,0,0,3,-0.188186,-0.190970,-0.701490,0.098453,-1.015506,-0.054984,0.329152,-0.965471,0.576635
4,0,0,4,-0.257462,-0.471325,-0.297420,0.074018,-0.324194,-0.597093,0.219856,-0.276356,-0.904790
5,0,0,5,0.027579,-0.020169,0.640348,-0.948373,-0.374251,-0.240350,-0.913801,-0.548867,-1.283726
6,0,0,6,-0.419646,-0.181228,-0.194079,0.667993,0.936857,0.517728,0.896325,1.068884,1.579290
7,0,0,7,-0.114118,-0.198511,-0.200027,-0.410021,-0.135167,-0.182887,-0.492168,-0.142915,-0.202081
8,0,0,8,-0.374147,0.092127,0.294723,0.402989,2.060188,-0.225042,0.956460,2.185598,-0.435856
9,0,0,9,-0.529529,0.040104,-0.333090,-0.959040,-1.318411,-0.774299,-0.716492,-1.471419,-1.107083


In [167]:
if os.path.isfile('submission.parquet'):
    pl_sub = pl.read_parquet('submission.parquet')
    print(len(pl_sub))
    display(pl_sub)

39


row_id,responder_6
i64,f64
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
…,…
34,0.0
35,0.0
36,0.0


In [61]:
model_eval

LSTMFeedForward(
  (lstm): LSTMEquavalentMLP(
    (input_gate): Linear(in_features=80, out_features=128, bias=True)
    (candidate_gate): Linear(in_features=80, out_features=128, bias=True)
    (output_gate): Linear(in_features=80, out_features=128, bias=True)
  )
  (fc): Linear(in_features=128, out_features=1, bias=True)
)

In [49]:
pl.read_parquet(os.getcwd() + '/input/test.parquet/date_id=0/part-0.parquet')


row_id,date_id,time_id,symbol_id,weight,is_scored,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,…,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78
i64,i16,i16,i8,f32,bool,f32,f32,f32,f32,f32,f32,f32,f32,f32,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0,0,0,0,3.169998,true,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,null,-0.0,null,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,0.0,-0.0,0.0,0.0,null,0.0,null,null,-0.0,null,-0.0,0.0,null,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,0.0,null,null,0.0,0.0,-0.0,-0.0
1,0,0,1,2.165993,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,null,-0.0,null,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,0.0,0.0,0.0,0.0,null,0.0,null,null,-0.0,null,-0.0,0.0,null,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,null,null,0.0,0.0,0.0,0.0
2,0,0,2,3.06555,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,null,-0.0,null,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,0.0,-0.0,-0.0,-0.0,null,0.0,null,null,-0.0,null,-0.0,0.0,null,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,null,null,0.0,0.0,-0.0,-0.0
3,0,0,3,2.698642,true,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,null,-0.0,null,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,0.0,-0.0,0.0,-0.0,null,-0.0,null,null,-0.0,null,-0.0,0.0,null,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,null,null,0.0,0.0,-0.0,-0.0
4,0,0,4,1.80333,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,null,-0.0,null,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,0.0,0.0,0.0,0.0,null,0.0,null,null,-0.0,null,-0.0,0.0,null,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,null,null,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
34,0,0,34,3.240565,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,null,-0.0,null,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,-0.0,-0.0,-0.0,-0.0,null,-0.0,null,null,-0.0,null,-0.0,-0.0,null,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,null,null,0.0,0.0,0.0,0.0
35,0,0,35,1.057221,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,null,-0.0,null,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,…,null,0.0,null,-0.0,0.0,0.0,0.0,0.0,null,0.0,null,null,0.0,null,-0.0,0.0,null,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,null,null,0.0,0.0,-0.0,-0.0
36,0,0,36,0.907022,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,null,-0.0,null,-0.0,-0.0,-0.0,null,-0.0,-0.0,0.0,-0.0,null,null,-0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,0.0,-0.0,0.0,0.0,null,0.0,null,null,0.0,null,-0.0,0.0,null,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,null,null,0.0,0.0,0.0,0.0


In [165]:
lags_ : pl.DataFrame | None = None


# Replace this function with your inference code.
# You can return either a Pandas or Polars dataframe, though Polars is recommended.
# Each batch of predictions (except the very first) must be returned within 1 minute of the batch features being provided.
def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    # All the responders from the previous day are passed in at time_id == 0. We save them in a global variable for access at every time_id.
    # Use them as extra features, if you like.
    global lags_
    if lags is not None:
        lags_ = lags

    # Replace this section with your own predictions
    predictions = test.select(
        'row_id',
        pl.lit(0.0).alias('responder_6'),
    )

    if isinstance(predictions, pl.DataFrame):
        assert predictions.columns == ['row_id', 'responder_6']
    elif isinstance(predictions, pd.DataFrame):
        assert (predictions.columns == ['row_id', 'responder_6']).all()
    else:
        raise TypeError('The predict function must return a DataFrame')
    # Confirm has as many rows as the test data.
    assert len(predictions) == len(test)

    return predictions



In [166]:
inference_server = js_server.JSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            test_parquet,
            lags_parquet
        )
    )


In [168]:
test_parquet

'/app/input/test.parquet'

In [169]:
pl.read_parquet(test_parquet)

row_id,date_id,time_id,symbol_id,weight,is_scored,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,…,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78
i64,i16,i16,i8,f32,bool,f32,f32,f32,f32,f32,f32,f32,f32,f32,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0,0,0,0,3.169998,true,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,null,-0.0,null,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,0.0,-0.0,0.0,0.0,null,0.0,null,null,-0.0,null,-0.0,0.0,null,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,0.0,null,null,0.0,0.0,-0.0,-0.0
1,0,0,1,2.165993,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,null,-0.0,null,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,0.0,0.0,0.0,0.0,null,0.0,null,null,-0.0,null,-0.0,0.0,null,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,null,null,0.0,0.0,0.0,0.0
2,0,0,2,3.06555,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,null,-0.0,null,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,0.0,-0.0,-0.0,-0.0,null,0.0,null,null,-0.0,null,-0.0,0.0,null,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,null,null,0.0,0.0,-0.0,-0.0
3,0,0,3,2.698642,true,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,null,-0.0,null,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,0.0,-0.0,0.0,-0.0,null,-0.0,null,null,-0.0,null,-0.0,0.0,null,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,null,null,0.0,0.0,-0.0,-0.0
4,0,0,4,1.80333,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,null,-0.0,null,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,0.0,0.0,0.0,0.0,null,0.0,null,null,-0.0,null,-0.0,0.0,null,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,null,null,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
34,0,0,34,3.240565,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,null,-0.0,null,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,-0.0,-0.0,-0.0,-0.0,null,-0.0,null,null,-0.0,null,-0.0,-0.0,null,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,null,null,0.0,0.0,0.0,0.0
35,0,0,35,1.057221,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,null,-0.0,null,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,…,null,0.0,null,-0.0,0.0,0.0,0.0,0.0,null,0.0,null,null,0.0,null,-0.0,0.0,null,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,null,null,0.0,0.0,-0.0,-0.0
36,0,0,36,0.907022,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,null,-0.0,null,-0.0,-0.0,-0.0,null,-0.0,-0.0,0.0,-0.0,null,null,-0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,0.0,-0.0,0.0,0.0,null,0.0,null,null,0.0,null,-0.0,0.0,null,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,null,null,0.0,0.0,0.0,0.0
